In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint
from IPython.display import clear_output
import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)

from matplotlib.collections import BrokenBarHCollection
import re

from structure_tools.Modules_tools import return_fsts

PCA_color_ref= ['darkseagreen','crimson', 'darkorange', 'darkblue', 'darkcyan',
            'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
            'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
            'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
            'darkslateblue', 'darkslategray', 'darkslategrey',
            'darkturquoise', 'darkviolet', 'deeppink']

## vcf analysis
Jupyter notebook for the local analysis of genetic data stored in .vcf format.

Perform analysis of structure across data set, followed by a more detailed study of variation across local genomic windows.

### Input

In [2]:
from structure_tools.vcf_geno_tools import simple_read_vcf

vcf_file= 'data_cleanRefs_simple_Admx.vcf'

genotype, summary, info_save= simple_read_vcf(vcf_file,row_info= 5,header_info= 9,phased= True)

print('Number of markers: {}'.format(genotype.shape[1]))
print('Number of individuals: {}'.format(genotype.shape[0]))

Number of markers: 40000
Number of individuals: 130


In [3]:
summary.head()


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT
0,1,8,1,A,T,.,PASS,.,GT:AD:DP
1,1,33,2,A,T,.,PASS,.,GT:AD:DP
2,1,74,3,A,T,.,PASS,.,GT:AD:DP
3,1,83,4,A,T,.,PASS,.,GT:AD:DP
4,1,87,5,A,T,.,PASS,.,GT:AD:DP


### Global variation

Perform PCA across data set.

Perform Mean shift clustering to attempt to extract genetically coherent groups of accessions.

These will later be used for supervised analysis.

In [4]:
from structure_tools.Tutorial_subplots import plot_global_pca

## Perform PCA
n_comp= 3
pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized')

feats= pca.fit_transform(genotype)

In [5]:
## perform MeanShift clustering.
bandwidth = estimate_bandwidth(feats, quantile=0.1)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=True, min_bin_freq=45)
ms.fit(feats)
labels1 = ms.labels_
label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1)))}
###

In [6]:
###
plot_global_pca(feats,label_select,PCA_color_ref,title= 'global_pca',height= 500,width= 950)

In [7]:
select_refs= [0,1,2]
label_vector= [[len(select_refs),labels1[x]][int(labels1[x] in select_refs)] for x in range(genotype.shape[0])]

Whose= list(range(genotype.shape[0]))


## Imputation

### Creating a matrix, introducing missing values


Single missing observation is introduced to a sample `nanObs`. Dimensionality reduction is applied to feature windows to generate a profile of distances between the `nanObs` and other samples. feature windows are selected randomly across the vcf, are free of missing data. 

For this notebook we will also keep window size constant. 


### I. nan_observation


In [17]:
print('full data set shape: {}'.format(genotype.shape))

nan_n= 1

xnan= np.random.randint(0,genotype.shape[1],size= nan_n)
ynan= np.random.randint(0,genotype.shape[0],size= nan_n)

nan_coords= [ynan,xnan]
nan_coords= np.array(nan_coords).T

print(nan_coords)


full data set shape: (130, 40000)
[[   41 39164]]


### Distance profiles.

for a single nan value, selected using `nan_idx` from the list `nan_coords` above.

define window size `wind_sizes` and number of windows `Nreps`. 

This version of the approach is calculating distances in feature space, following dimensionality redution (Dr). PCA is used here. 

Define:
- `ncomps` : the number of dimensions to retain during Dr. 
- `dimN`: number of dimensions used to perfiorm distance calculations.
- `metric`: metric to use in distance calculations. default= eucledian. 



In [18]:
from impute_tools.impute_tools_I import rand_wdDist

nan_idx= 0
nan_obs= nan_coords[nan_idx]
nan_acc= nan_obs[0]
nan_pos= nan_obs[1]

wind_sizes= 100
Nreps= 400
ncomps= 5
dimN= 2
metric= 'euclidean'

dist_store= rand_wdDist(genotype, nan_coords,
               wind_sizes= 100,
                Nreps= 400,
                ncomps= 5,
                dimN= 2,
                metric= 'euclidean')
dist_store.shape

(400, 129)

### Distance summary stats

#### A. observation distance variance.

In [19]:

dist_var= np.std(dist_store,axis= 0)**2
who_plot= [x for x in range(len(label_vector)) if label_vector[x] in [0,1,2,3]]
idvector= ['pop{}_{}'.format(label_vector[x],Whose[x]) for x in who_plot]

fig = go.Figure([go.Bar(
    x=idvector, y=dist_var
)])
layout= go.Layout()

Figure= go.Figure(data= fig, layout= layout)
iplot(Figure)


#### B. Coordinates at local window.

Dimensionality reduction of features surrounding missing call. Nfeatures=  wind_sizes.

Coordinates of observation with missing data indicated with red circle.



In [20]:
local_l= genotype[:,(nan_pos-int(wind_sizes/2)):(nan_pos+int(wind_sizes/2))]
coords= {z:[x for x in range(len(label_vector)) if label_vector[x] == z] for z in list(set(label_vector))}

pca2 = PCA(n_components=ncomps, whiten=False,svd_solver='randomized')
featl= pca2.fit_transform(local_l)

figwl= [go.Scatter(
    x= featl[coords[i],0],
    y= featl[coords[i],1],
    mode= 'markers',
    name= str(i)
) for i in coords.keys()]

figwl.append(go.Scatter(
    mode='markers',
    x=[featl[nan_acc,0]],
    y=[featl[nan_acc,1]],
    marker=dict(
        color='rgba(135, 206, 250, 0)',
        size=25,
        opacity= 1,
        line=dict(
            color='red',
            width=5
        )
    ),
    showlegend=False
))

layout= go.Layout()

Figure_wl= go.Figure(data= figwl, layout= layout)

iplot(Figure_wl)


### Distance profile - reference set.

Decide which obervations to use the distance to, and where, to predict `nanObs` coordinates. 

**context**: different windows might span different patterns of admixture / structure. In the case the sample carrying the missing call is admixed, distance profiles may differ considerably. there are two main concerns here:
- other admixed: we would like to reduce or remove the weight of the distances to these.
- admixture border: `nanObs` is itself admixed, resulting in significantly different distance profiles. 

Observation specific distances across are measured across trainning windows. 

The approach below proposes to tackle both concerns raised above:

    i. Variance in distance is calculated at the individual level. We seek the least variable reference points.
    Variance clusters are identified using mean shift.
    ii. Different distance profile groups identified using MeanShift. allows single cluster and excludes outliers.

The group of accessions with the smallest average std is selected as focus obs.

Reference distance profiles are selected as the largest cluster of profiles.  

#### A. Distance profiles. 


Distance profiles consist of the ordered vectors of distances `nanObs` / `every other obs` of length N-1.

Dimensionality reduction is applied and distances calculated in the resulting feature space. 

Here using `PCA` and `euclidean disances`.



In [21]:

pca2 = PCA(n_components=ncomps, whiten=False,svd_solver='randomized')
featd= pca2.fit_transform(dist_store)
bandwidth = estimate_bandwidth(featd, quantile=0.25)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=False, min_bin_freq=35)
ms.fit(featd)
labelsf = ms.labels_
labelf_select = {y:[x for x in range(len(labelsf)) if labelsf[x] == y] for y in sorted(list(set(labelsf)))}


fig= [go.Scatter(
    x= featd[labelf_select[i],0],
    y= featd[labelf_select[i],1],
    mode= 'markers',
    name= str(i)
) for i in labelf_select.keys()]

layout= go.Layout()

Figure= go.Figure(data= fig, layout= layout)

iplot(Figure)



#### B. Variance across data sets. 

Accession subset: `std_gp_use`


In [22]:
# subselect by variance in distances:

## perform MeanShift clustering.
#bandwidth = estimate_bandwidth(featl, quantile=0.2)
bandwidth = estimate_bandwidth(dist_var.reshape(-1,1), quantile=0.2)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=False, min_bin_freq=35)

ms.fit(dist_var.reshape(-1,1))
labels_std = ms.labels_
std_select = {y:[x for x in range(len(labels_std)) if labels_std[x] == y] for y in sorted(list(set(labels_std)))}
std_select.keys()

std_gpmeans= {z: np.mean([dist_var[x] for x in g]) for z,g in std_select.items() if z != -1}

std_gp_use= sorted(std_gpmeans,key= std_gpmeans.get)
d= 0
idx = 0

while d != 1:
    g=std_select[std_gp_use[idx]] 
    
    if len(g) >= 15:
        std_gp_use= list(g)
        d= 1
        
    idx+= 1

#std_gp_use= std_select[std_gp_use]

#std_gp_use= [other_obs[x] for x in std_gp_use]
#std_gp_use= list(range(len(labels_std)))

### Coordinate inference

**Grid approach**

Produce grid encompassing sample coordinates in the feature space of the window covering the nan value. 

#### I. Grid construction:

- P: grid mesh.
- dimN: number of features from Quanted_set to use. 



In [25]:
from impute_tools.impute_tools_II import get_bg_grid

Quanted_set= np.array(featl)
P= 20
dimN= 2

background= get_bg_grid(Quanted_set, P= P, dimN= dimN)

#### II. Grid distances.

Create distance profile for every position in `background` grid.


In [27]:
from sklearn.metrics import pairwise_distances

workfeat= featl[std_gp_use,:dimN]

dist_grid= pairwise_distances(background, workfeat,
                                            metric=metric)

dist_grid.shape

(400, 19)

#### III. Coordinate likelihood.

    i. BG and reference distance vectors go through Dr together (`PCA` here). Kernel Density Estimation is fit to reference obs coordinates. Likelihoods are extracted for BG coordinate positions. 

    ii. Resulting likelihoods are mapped to the feature space capturing `nanObs`.


In [28]:
from impute_tools.impute_tools import kde_likes_extract

dist_comps= 10
Bandwidth_split = 30

####
####
dist_ref_select= 0
dist_ref= dist_store[labelf_select[dist_ref_select],:]
dist_ref= dist_ref[:,std_gp_use]


grid_likes= kde_likes_extract(dist_grid,dist_ref)

In [30]:
from plotly import tools

title= 'coords'
fig_subplots = tools.make_subplots(rows=1, cols=2,subplot_titles=tuple([title]*2))

for trace in figwl:
    fig_subplots.append_trace(trace, 1, 1)
    

trace= go.Scatter(
    x= background[:,0],
    y= background[:,1],
    #z= grid_likes,
    mode= 'markers',
    marker= {
        'color':grid_likes,
        'colorbar': go.scatter.marker.ColorBar(
            title= 'ColorBar'
        ),
        'colorscale':'Viridis',
        'line': {'width': 0},
        'size': 25,
        'symbol': 'circle',
      "opacity": 1
      }
)

fig_subplots.append_trace(trace, 1,2)

iplot(fig_subplots)

c:\users\floyd\appdata\local\programs\python\python37-32\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



### IV. Grid walk.

Grids can quickly become very heavy (grid edge length times dimensions). We propose to narrow down the search progressively.

Observations with high likelihood are used to reproduce background coordinates within their distribution in successive layers of analysis.



In [38]:
from impute_tools.impute_tools_VI import (
    nBg_MS, nBg_grid,
    gridWalk
)

dist_ref_select= 0
dist_ref= dist_store[labelf_select[dist_ref_select],:]
dist_ref= dist_ref[:,std_gp_use]

P= 20
dimN= 2
N_samps= P**dimN
dist_comps= 10
Bandwidth_split = 30
kernel= 'gaussian'



BG_func= nBg_MS
BG_args= {
    'lb':0.05,
    'up':0.8,
    'kernel': kernel,
    'N_samps': N_samps
}


granted, grid_likes= gridWalk(featl,dist_ref,BG_func, BG_args= BG_args, std_gp_use= std_gp_use,
            P= P,
            dimN= dimN,
            N_samps= N_samps,
            dist_comps= dist_comps,
            Bandwidth_split = Bandwidth_split,
            metric= metric,
            kernel= kernel)



In [39]:
from plotly import tools

title= 'coords'
fig_subplots = tools.make_subplots(rows=1, cols=2,subplot_titles=tuple([title]*2))

for trace in figwl:
    fig_subplots.append_trace(trace, 1, 1)
    

trace= go.Scatter(
    x= granted[:,0],
    y= granted[:,1],
    #z= grid_likes,
    mode= 'markers',
    marker= {
        'color':grid_likes,
        'colorbar': go.scatter.marker.ColorBar(
            title= 'ColorBar'
        ),
        'colorscale':'Viridis',
        'line': {'width': 0},
        'size': 5,
        'symbol': 'circle',
      "opacity": 1
      }
)

fig_subplots.append_trace(trace, 1,2)

iplot(fig_subplots)

c:\users\floyd\appdata\local\programs\python\python37-32\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

